# Smart DB Connector V3 - Constraints Testing (FINAL)

Testing database constraints with our Smart DB Connector class:
1. ✅ Connect to both databases using db_connector
2. ✅ CREATE TABLE with PRIMARY KEY and FOREIGN KEY constraints 
3. ✅ Use populate() function with mode='append'
4. ✅ Test constraints functionality

**Logic**: Use district_id if available (preferred), otherwise use district for FK

In [1]:
# Import our Smart DB Connector
import pandas as pd
import sys
from pathlib import Path
import time
import random

# Add connector path
current_dir = Path.cwd()
if current_dir.name == 'tests':
    parent_dir = current_dir.parent
else:
    parent_dir = Path("/Users/svitlanakovalivska/layered-populate-data-pool-da/db_population_utils/db_connector")

sys.path.insert(0, str(parent_dir))

# Import our connector
from smart_db_connector_enhanced_V3 import db_connector
print("✅ Smart DB Connector V3 imported successfully")

✅ Smart DB Connector V3 imported successfully


In [2]:
# Connect to NeonDB
print("🔌 CONNECTING TO NEONDB")
print("=" * 30)

neon_db = db_connector()  # Default NeonDB connection
print(f"✅ NeonDB connected: {neon_db.connection_type}")
print(f"📋 Schema: {neon_db.current_schema}")
print(f"📊 Tables: {len(neon_db.tables)}")

🔌 CONNECTING TO NEONDB
🌟 SMART DATABASE CONNECTOR V3 - INITIALIZING...
🔗 Using default NeonDB connection
✅ NeonDB configuration loaded
   Default schema: test_berlin_data
🔌 Connecting to NeonDB...
✅ Connection successful!
   Database: neondb
   User: neondb_owner

🔍 Auto-discovering database schemas...
✅ Discovered 4 schemas
🎯 Auto-selected default schema: test_berlin_data

📊 SMART DB CONNECTOR V3 - CONNECTION SUMMARY
🔗 Connection Type: NeonDB

🗂️  Discovered 4 schemas:
  📁 dependency_example: 5 tables
       └─ banks_test_kovalivska_aws (11 columns)
       └─ departments (2 columns)
       └─ districts (3 columns)
       └─ ... and 2 more tables
  📁 nyc_schools: 27 tables
       └─ Audrey_sat_results (10 columns)
       └─ Colleges_Berlin (12 columns)
       └─ Levon_cleaned_sat_scores (8 columns)
       └─ ... and 24 more tables
  📁 public: 15 tables
       └─ audrey_sat_results (10 columns)
       └─ cleaned_sat_results_peter_s (9 columns)
       └─ demo_users (6 columns)
       └─ 

In [3]:
# Connect to AWS LayeredDB
print("🔌 CONNECTING TO AWS LAYEREDDB")
print("=" * 35)

username = 'svitlana_kovalivska'
password = '4i3mRyKE38edL3'

try:
    aws_db = db_connector(database='layereddb', username=username, password=password)
    print(f"✅ AWS LayeredDB connected: {aws_db.connection_type}")
    print(f"📋 Schema: {aws_db.current_schema}")
    print(f"📊 Tables: {len(aws_db.tables)}")
    aws_connected = True
except Exception as e:
    print(f"❌ AWS connection failed: {e}")
    print("Will continue with NeonDB only")
    aws_connected = False

🔌 CONNECTING TO AWS LAYEREDDB
🌟 SMART DATABASE CONNECTOR V3 - INITIALIZING...
🚇 AWS LayeredDB connection requested
🚇 Tunnel Status: Connected
✅ AWS LayeredDB configuration loaded
   Tunnel: Tunnel is active on localhost:5433
🔌 Connecting to AWS LayeredDB...
✅ Connection successful!
   Database: layereddb
   User: svitlana_kovalivska

🔍 Auto-discovering database schemas...
✅ Discovered 2 schemas
🎯 Auto-selected default schema: berlin_source_data

📊 SMART DB CONNECTOR V3 - CONNECTION SUMMARY
🔗 Connection Type: AWS LayeredDB
🚇 Tunnel Status: Connected (localhost:5433)

🗂️  Discovered 2 schemas:
  🎯 [CURRENT] berlin_source_data: 10 tables
       └─ aws_test_customers_v3 (5 columns)
       └─ aws_test_products_v3 (6 columns)
       └─ aws_test_sales_v3 (6 columns)
       └─ ... and 7 more tables
  📁 public: 22 tables
       └─ aws_test_customers_v3 (5 columns)
       └─ aws_test_products_v3 (6 columns)
       └─ aws_test_sales_v3 (6 columns)
       └─ ... and 19 more tables

💡 Quick Command

In [4]:
# Analyze districts table structures and get valid FK values
print("🔍 ANALYZING DISTRICTS TABLES")
print("=" * 30)

# NeonDB districts analysis
print("📗 NeonDB Districts:")
neon_districts_info = neon_db.query("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'test_berlin_data' AND table_name = 'districts'
    ORDER BY ordinal_position
""", show_info=False)
print(neon_districts_info)

# Determine NeonDB FK column and values
if 'district_id' in neon_districts_info['column_name'].values:
    neon_fk_column = 'district_id'
    neon_districts = neon_db.query("SELECT district_id FROM districts ORDER BY district_id LIMIT 5", show_info=False)
    neon_valid_districts = neon_districts['district_id'].tolist()
    print(f"🔑 NeonDB FK: district_id, values: {neon_valid_districts}")
elif 'district' in neon_districts_info['column_name'].values:
    neon_fk_column = 'district'
    neon_districts = neon_db.query("SELECT district FROM districts ORDER BY district LIMIT 5", show_info=False)
    neon_valid_districts = neon_districts['district'].tolist()
    print(f"🔑 NeonDB FK: district, values: {neon_valid_districts[:3]}...")
else:
    neon_fk_column = 'district'
    neon_valid_districts = ['Mitte', 'Pankow', 'Charlottenburg-Wilmersdorf']
    print(f"🔑 NeonDB FK: district (fallback), values: {neon_valid_districts}")

# AWS districts analysis (if connected)
if aws_connected:
    print("\n📘 AWS Districts:")
    aws_districts_info = aws_db.query("""
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_schema = 'berlin_source_data' AND table_name = 'districts'
        ORDER BY ordinal_position
    """, show_info=False)
    print(aws_districts_info)
    
    if 'district_id' in aws_districts_info['column_name'].values:
        aws_fk_column = 'district_id'
        aws_districts = aws_db.query("SELECT district_id FROM districts ORDER BY district_id LIMIT 5", show_info=False)
        aws_valid_districts = aws_districts['district_id'].tolist()
        print(f"🔑 AWS FK: district_id, values: {aws_valid_districts}")
    elif 'district' in aws_districts_info['column_name'].values:
        aws_fk_column = 'district'
        aws_districts = aws_db.query("SELECT district FROM districts ORDER BY district LIMIT 5", show_info=False)
        aws_valid_districts = aws_districts['district'].tolist()
        print(f"🔑 AWS FK: district, values: {aws_valid_districts[:3]}...")
    else:
        aws_fk_column = 'district_id'
        aws_valid_districts = ['11001001', '11002002', '11003003']
        print(f"🔑 AWS FK: district_id (fallback), values: {aws_valid_districts}")
else:
    aws_fk_column = neon_fk_column
    aws_valid_districts = neon_valid_districts

print(f"\n🎯 Summary: NeonDB uses {neon_fk_column}, AWS uses {aws_fk_column}")

🔍 ANALYZING DISTRICTS TABLES
📗 NeonDB Districts:
    column_name          data_type
0      district  character varying
1      geometry       USER-DEFINED
2  geometry_str               text
🔑 NeonDB FK: district, values: ['Charlottenburg-Wilmersdorf', 'Friedrichshain-Kreuzberg', 'Lichtenberg']...

📘 AWS Districts:
   column_name          data_type
0  district_id  character varying
1     district  character varying
2     geometry       USER-DEFINED
🔑 AWS FK: district_id, values: ['11001001', '11002002', '11003003', '11004004', '11005005']

🎯 Summary: NeonDB uses district, AWS uses district_id


In [5]:
# Create test data
print("📊 CREATING TEST DATA")
print("=" * 20)

test_data = pd.DataFrame({
    'bank_id': ['SMART001', 'SMART002', 'SMART003'],
    'name': ['Smart Bank 1', 'Smart Bank 2', 'Smart Bank 3'],
    'district_id': neon_valid_districts[:3],
    'address': ['Address 1', 'Address 2', 'Address 3'],
    'phone': ['+49 30 111111', '+49 30 222222', '+49 30 333333']
})

print("✅ Test data created:")
print(test_data)
print(f"🔑 Using {neon_fk_column} values: {test_data['district_id'].tolist()}")

📊 CREATING TEST DATA
✅ Test data created:
    bank_id          name                 district_id    address  \
0  SMART001  Smart Bank 1  Charlottenburg-Wilmersdorf  Address 1   
1  SMART002  Smart Bank 2    Friedrichshain-Kreuzberg  Address 2   
2  SMART003  Smart Bank 3                 Lichtenberg  Address 3   

           phone  
0  +49 30 111111  
1  +49 30 222222  
2  +49 30 333333  
🔑 Using district values: ['Charlottenburg-Wilmersdorf', 'Friedrichshain-Kreuzberg', 'Lichtenberg']


In [6]:
# STEP 1: Create NeonDB table with constraints
print("🏗️ STEP 1: CREATE NEONDB TABLE WITH CONSTRAINTS")
print("=" * 50)

# Generate unique table name
table_suffix = f"{int(time.time()) % 10000}_{random.randint(1000, 9999)}"
table_name = f"smart_banks_test_{table_suffix}"

print(f"📋 Table: {table_name}")
print(f"🔑 FK column: {neon_fk_column}")

create_neon_sql = f"""
CREATE TABLE test_berlin_data.{table_name} (
    bank_id VARCHAR(20) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    district_id TEXT,
    address TEXT,
    phone VARCHAR(20),
    CONSTRAINT fk_{table_name}_district 
        FOREIGN KEY (district_id) 
        REFERENCES test_berlin_data.districts({neon_fk_column})
        ON DELETE RESTRICT
);
"""

try:
    from sqlalchemy import text
    
    # Method 1: Explicit transaction
    with neon_db.engine.connect() as conn:
        trans = conn.begin()
        try:
            conn.execute(text(create_neon_sql))
            trans.commit()
            print("✅ NeonDB table created (Method 1)")
        except Exception as e1:
            trans.rollback()
            print(f"⚠️ Method 1 failed: {str(e1)[:100]}...")
            
            # Method 2: Autocommit
            try:
                conn.execution_options(autocommit=True).execute(text(create_neon_sql))
                print("✅ NeonDB table created (Method 2)")
            except Exception as e2:
                print(f"❌ Method 2 failed: {str(e2)[:100]}...")
                raise e2
    
    # Verify table and constraints
    time.sleep(2)
    
    count_result = neon_db.query(f"SELECT COUNT(*) as count FROM test_berlin_data.{table_name}", show_info=False)
    print(f"✅ Table verified: {count_result['count'].iloc[0]} rows")
    
    constraints = neon_db.query(f"""
        SELECT constraint_name, constraint_type 
        FROM information_schema.table_constraints 
        WHERE table_schema = 'test_berlin_data' AND table_name = '{table_name}'
    """, show_info=False)
    
    print(f"🔒 Constraints: {len(constraints)}")
    for _, c in constraints.iterrows():
        print(f"   ✅ {c['constraint_type']}: {c['constraint_name']}")
        
    neon_table_created = True
    
except Exception as e:
    print(f"❌ NeonDB table creation failed: {e}")
    neon_table_created = False

🏗️ STEP 1: CREATE NEONDB TABLE WITH CONSTRAINTS
📋 Table: smart_banks_test_6802_9949
🔑 FK column: district
✅ NeonDB table created (Method 1)
✅ Table verified: 0 rows
🔒 Constraints: 4
   ✅ PRIMARY KEY: smart_banks_test_6802_9949_pkey
   ✅ FOREIGN KEY: fk_smart_banks_test_6802_9949_district
   ✅ CHECK: 204800_1843224_1_not_null
   ✅ CHECK: 204800_1843224_2_not_null


In [7]:
# Create AWS table if connected
if aws_connected:
    print(f"\n🏗️ CREATING AWS TABLE: {table_name}")
    print("=" * 35)
    print(f"🔑 AWS FK column: {aws_fk_column}")
    
    create_aws_sql = f"""
    CREATE TABLE berlin_source_data.{table_name} (
        bank_id VARCHAR(20) PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        district_id TEXT,
        address TEXT,
        phone VARCHAR(20),
        CONSTRAINT fk_{table_name}_district 
            FOREIGN KEY (district_id) 
            REFERENCES berlin_source_data.districts({aws_fk_column})
            ON DELETE RESTRICT
    );
    """
    
    try:
        with aws_db.engine.connect() as conn:
            trans = conn.begin()
            try:
                conn.execute(text(create_aws_sql))
                trans.commit()
                print("✅ AWS table created (Method 1)")
            except Exception as e1:
                trans.rollback()
                print(f"⚠️ AWS Method 1 failed: {str(e1)[:100]}...")
                
                try:
                    conn.execution_options(autocommit=True).execute(text(create_aws_sql))
                    print("✅ AWS table created (Method 2)")
                except Exception as e2:
                    print(f"❌ AWS Method 2 failed: {str(e2)[:100]}...")
                    raise e2
        
        time.sleep(2)
        count_result = aws_db.query(f"SELECT COUNT(*) as count FROM berlin_source_data.{table_name}", show_info=False)
        print(f"✅ AWS table verified: {count_result['count'].iloc[0]} rows")
        
        constraints = aws_db.query(f"""
            SELECT constraint_name, constraint_type 
            FROM information_schema.table_constraints 
            WHERE table_schema = 'berlin_source_data' AND table_name = '{table_name}'
        """, show_info=False)
        
        print(f"🔒 AWS Constraints: {len(constraints)}")
        for _, c in constraints.iterrows():
            print(f"   ✅ {c['constraint_type']}: {c['constraint_name']}")
            
        aws_table_created = True
        
    except Exception as e:
        print(f"❌ AWS table creation failed: {str(e)[:150]}...")
        aws_table_created = False
else:
    aws_table_created = False


🏗️ CREATING AWS TABLE: smart_banks_test_6802_9949
🔑 AWS FK column: district_id
✅ AWS table created (Method 1)
✅ AWS table verified: 0 rows
🔒 AWS Constraints: 4
   ✅ PRIMARY KEY: smart_banks_test_6802_9949_pkey
   ✅ FOREIGN KEY: fk_smart_banks_test_6802_9949_district
   ✅ CHECK: 24602_57956_1_not_null
   ✅ CHECK: 24602_57956_2_not_null


## STEP 2: Populate Tables Using Smart DB Connector

Use our connector's `populate()` function with `mode='append'` to preserve constraints

In [8]:
# STEP 2: Populate NeonDB table
print("📊 STEP 2: POPULATE TABLES")
print("=" * 30)

if neon_table_created:
    print("📗 NEONDB POPULATION:")
    try:
        result = neon_db.populate(
            df=test_data,
            table_name=table_name,
            schema='test_berlin_data',
            mode='append',  # ✅ PRESERVE CONSTRAINTS
            show_report=True
        )
        
        print(f"✅ NeonDB result: {result['status']}")
        if result['status'] == 'success':
            print(f"📊 Rows: {result.get('rows_inserted', 0)}")
            
            data_check = neon_db.query(f"SELECT * FROM test_berlin_data.{table_name}", show_info=False)
            print(f"📋 Data: {len(data_check)} rows")
            print(data_check[['bank_id', 'name', 'district_id']])
            
        neon_populated = True
        
    except Exception as e:
        print(f"❌ NeonDB population failed: {e}")
        neon_populated = False
else:
    print("⚠️ NeonDB table not created")
    neon_populated = False

📊 STEP 2: POPULATE TABLES
📗 NEONDB POPULATION:

📊 SMART POPULATE - PRE-POPULATION ANALYSIS
🎯 Target: test_berlin_data.smart_banks_test_6802_9949
📝 Mode: APPEND
🔗 Connection: ConnectionType.NEON_DB

📋 DATASET ANALYSIS:
   Rows: 3
   Columns: 5
   Memory usage: 0.00 MB

🔍 COLUMN ANALYSIS:
   bank_id: object | Nulls: 0 (0.0%) | Unique: 3
   name: object | Nulls: 0 (0.0%) | Unique: 3
   district_id: object | Nulls: 0 (0.0%) | Unique: 3
   address: object | Nulls: 0 (0.0%) | Unique: 3
   phone: object | Nulls: 0 (0.0%) | Unique: 3

✅ DATA QUALITY CHECKS:
   Total null values: 0
   Duplicate rows: 0

🏗️  TABLE STATUS:
   Table exists: No
📝 Inserting 3 rows × 5 columns
   Target: test_berlin_data.smart_banks_test_6802_9949
   Action: append
✅ Insert completed successfully

🎉 SMART POPULATE - OPERATION COMPLETED
✅ Status: SUCCESS
🎯 Table: test_berlin_data.smart_banks_test_6802_9949
📝 Mode: APPEND
⏱️  Execution time: 1.03 seconds
📊 Rows processed: 3
⚡ Performance: 3 rows/second

✅ NeonDB result

In [9]:
# Populate AWS table
if aws_table_created:
    print("\n📘 AWS POPULATION:")
    try:
        # Use AWS district values
        aws_test_data = test_data.copy()
        aws_test_data['district_id'] = aws_valid_districts[:3]
        
        print(f"🔑 AWS district values: {aws_test_data['district_id'].tolist()}")
        
        result = aws_db.populate(
            df=aws_test_data,
            table_name=table_name,
            schema='berlin_source_data',
            mode='append',
            show_report=True
        )
        
        print(f"✅ AWS result: {result['status']}")
        if result['status'] == 'success':
            print(f"📊 Rows: {result.get('rows_inserted', 0)}")
            
            data_check = aws_db.query(f"SELECT * FROM berlin_source_data.{table_name}", show_info=False)
            print(f"📋 Data: {len(data_check)} rows")
            print(data_check[['bank_id', 'name', 'district_id']])
            
        aws_populated = True
        
    except Exception as e:
        print(f"❌ AWS population failed: {e}")
        aws_populated = False
else:
    print("⚠️ AWS table not created")
    aws_populated = False


📘 AWS POPULATION:
🔑 AWS district values: ['11001001', '11002002', '11003003']

📊 SMART POPULATE - PRE-POPULATION ANALYSIS
🎯 Target: berlin_source_data.smart_banks_test_6802_9949
📝 Mode: APPEND
🔗 Connection: ConnectionType.AWS_LAYERED_DB

📋 DATASET ANALYSIS:
   Rows: 3
   Columns: 5
   Memory usage: 0.00 MB

🔍 COLUMN ANALYSIS:
   bank_id: object | Nulls: 0 (0.0%) | Unique: 3
   name: object | Nulls: 0 (0.0%) | Unique: 3
   district_id: object | Nulls: 0 (0.0%) | Unique: 3
   address: object | Nulls: 0 (0.0%) | Unique: 3
   phone: object | Nulls: 0 (0.0%) | Unique: 3

✅ DATA QUALITY CHECKS:
   Total null values: 0
   Duplicate rows: 0

🏗️  TABLE STATUS:
   Table exists: No
📝 Inserting 3 rows × 5 columns
   Target: berlin_source_data.smart_banks_test_6802_9949
   Action: append
✅ Insert completed successfully

🎉 SMART POPULATE - OPERATION COMPLETED
✅ Status: SUCCESS
🎯 Table: berlin_source_data.smart_banks_test_6802_9949
📝 Mode: APPEND
⏱️  Execution time: 0.26 seconds
📊 Rows processed: 3


## STEP 3: Test Constraints Functionality

Test that our constraints actually work by trying to violate them

In [10]:
# STEP 3: Test constraints
print("🧪 STEP 3: TEST CONSTRAINTS")
print("=" * 30)

if neon_populated:
    print("1️⃣ NeonDB Primary Key Test:")
    
    # Try duplicate PK
    duplicate_data = pd.DataFrame({
        'bank_id': ['SMART001'],  # Duplicate!
        'name': ['Duplicate Bank'],
        'district_id': [neon_valid_districts[0]],
        'address': ['Duplicate Address'],
        'phone': ['+49 30 999999']
    })
    
    try:
        result = neon_db.populate(
            df=duplicate_data,
            table_name=table_name,
            schema='test_berlin_data',
            mode='append',
            show_report=False
        )
        
        if result['status'] == 'error' and ('duplicate' in result.get('error', '').lower() or 'unique' in result.get('error', '').lower()):
            print("   ✅ Primary Key working")
        else:
            print("   ❌ Primary Key NOT working")
            
    except Exception as e:
        if 'duplicate' in str(e).lower() or 'unique' in str(e).lower():
            print("   ✅ Primary Key working")
        else:
            print(f"   ⚠️ Unexpected PK error: {str(e)[:100]}...")
    
    print("\n2️⃣ NeonDB Foreign Key Test:")
    
    # Try invalid FK
    invalid_fk_data = pd.DataFrame({
        'bank_id': ['INVALID001'],
        'name': ['Invalid FK Bank'],
        'district_id': ['NONEXISTENT_DISTRICT'],
        'address': ['Invalid Address'],
        'phone': ['+49 30 000000']
    })
    
    try:
        result = neon_db.populate(
            df=invalid_fk_data,
            table_name=table_name,
            schema='test_berlin_data',
            mode='append',
            show_report=False
        )
        
        if result['status'] == 'error' and 'foreign key' in result.get('error', '').lower():
            print("   ✅ Foreign Key working")
        else:
            print("   ❌ Foreign Key NOT working")
            
    except Exception as e:
        if 'foreign key' in str(e).lower() or 'violates' in str(e).lower():
            print("   ✅ Foreign Key working")
        else:
            print(f"   ⚠️ Unexpected FK error: {str(e)[:100]}...")
else:
    print("⚠️ NeonDB not populated, skipping tests")

🧪 STEP 3: TEST CONSTRAINTS
1️⃣ NeonDB Primary Key Test:
📝 Inserting 1 rows × 5 columns
   Target: test_berlin_data.smart_banks_test_6802_9949
   Action: append
❌ Insert operation failed: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "smart_banks_test_6802_9949_pkey"
DETAIL:  Key (bank_id)=(SMART001) already exists.

[SQL: INSERT INTO test_berlin_data.smart_banks_test_6802_9949 (bank_id, name, district_id, address, phone) VALUES (%(bank_id_m0)s, %(name_m0)s, %(district_id_m0)s, %(address_m0)s, %(phone_m0)s)]
[parameters: {'bank_id_m0': 'SMART001', 'name_m0': 'Duplicate Bank', 'district_id_m0': 'Charlottenburg-Wilmersdorf', 'address_m0': 'Duplicate Address', 'phone_m0': '+49 30 999999'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
   ✅ Primary Key working

2️⃣ NeonDB Foreign Key Test:
📝 Inserting 1 rows × 5 columns
   Target: test_berlin_data.smart_banks_test_6802_9949
   Action: append
❌ Insert operation failed: (psycopg2.errors.Foreig

In [11]:
# Test AWS constraints if available
if aws_populated:
    print("\n3️⃣ AWS Constraint Tests:")
    
    # Primary Key Test
    print("   Primary Key Test:")
    duplicate_data = pd.DataFrame({
        'bank_id': ['SMART001'],  # Duplicate!
        'name': ['AWS Duplicate'],
        'district_id': [aws_valid_districts[0]],
        'address': ['AWS Duplicate'],
        'phone': ['+49 30 999999']
    })
    
    try:
        result = aws_db.populate(
            df=duplicate_data,
            table_name=table_name,
            schema='berlin_source_data',
            mode='append',
            show_report=False
        )
        
        if result['status'] == 'error' and ('duplicate' in result.get('error', '').lower() or 'unique' in result.get('error', '').lower()):
            print("      ✅ AWS Primary Key working")
        else:
            print("      ❌ AWS Primary Key NOT working")
            
    except Exception as e:
        if 'duplicate' in str(e).lower():
            print("      ✅ AWS Primary Key working")
        else:
            print(f"      ⚠️ AWS PK error: {str(e)[:80]}...")
    
    # Foreign Key Test
    print("   Foreign Key Test:")
    invalid_fk_data = pd.DataFrame({
        'bank_id': ['AWS_INVALID'],
        'name': ['AWS Invalid FK'],
        'district_id': ['INVALID_AWS_DISTRICT'],
        'address': ['AWS Invalid'],
        'phone': ['+49 30 000000']
    })
    
    try:
        result = aws_db.populate(
            df=invalid_fk_data,
            table_name=table_name,
            schema='berlin_source_data',
            mode='append',
            show_report=False
        )
        
        if result['status'] == 'error' and 'foreign key' in result.get('error', '').lower():
            print("      ✅ AWS Foreign Key working")
        else:
            print("      ❌ AWS Foreign Key NOT working")
            
    except Exception as e:
        if 'foreign key' in str(e).lower():
            print("      ✅ AWS Foreign Key working")
        else:
            print(f"      ⚠️ AWS FK error: {str(e)[:80]}...")
else:
    print("⚠️ AWS not populated, skipping tests")


3️⃣ AWS Constraint Tests:
   Primary Key Test:
📝 Inserting 1 rows × 5 columns
   Target: berlin_source_data.smart_banks_test_6802_9949
   Action: append
❌ Insert operation failed: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "smart_banks_test_6802_9949_pkey"
DETAIL:  Key (bank_id)=(SMART001) already exists.

[SQL: INSERT INTO berlin_source_data.smart_banks_test_6802_9949 (bank_id, name, district_id, address, phone) VALUES (%(bank_id_m0)s, %(name_m0)s, %(district_id_m0)s, %(address_m0)s, %(phone_m0)s)]
[parameters: {'bank_id_m0': 'SMART001', 'name_m0': 'AWS Duplicate', 'district_id_m0': '11001001', 'address_m0': 'AWS Duplicate', 'phone_m0': '+49 30 999999'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
      ✅ AWS Primary Key working
   Foreign Key Test:
📝 Inserting 1 rows × 5 columns
   Target: berlin_source_data.smart_banks_test_6802_9949
   Action: append
❌ Insert operation failed: (psycopg2.errors.ForeignKeyViolation) insert or up

## STEP 4: Final Verification

Check that constraints are still in place after all operations

In [12]:
# STEP 4: Final verification
print("🎯 STEP 4: FINAL VERIFICATION")
print("=" * 30)

if neon_populated:
    print("📗 NeonDB Final Status:")
    
    constraints = neon_db.query(f"""
        SELECT constraint_name, constraint_type 
        FROM information_schema.table_constraints 
        WHERE table_schema = 'test_berlin_data' AND table_name = '{table_name}'
        ORDER BY constraint_type
    """, show_info=False)
    
    print(f"🔒 Constraints after all operations: {len(constraints)}")
    for _, c in constraints.iterrows():
        print(f"   ✅ {c['constraint_type']}: {c['constraint_name']}")
    
    final_count = neon_db.query(f"SELECT COUNT(*) as count FROM test_berlin_data.{table_name}", show_info=False)
    print(f"📊 Final rows: {final_count['count'].iloc[0]}")

if aws_populated:
    print("\n📘 AWS Final Status:")
    
    constraints = aws_db.query(f"""
        SELECT constraint_name, constraint_type 
        FROM information_schema.table_constraints 
        WHERE table_schema = 'berlin_source_data' AND table_name = '{table_name}'
        ORDER BY constraint_type
    """, show_info=False)
    
    print(f"🔒 Constraints after all operations: {len(constraints)}")
    for _, c in constraints.iterrows():
        print(f"   ✅ {c['constraint_type']}: {c['constraint_name']}")
    
    final_count = aws_db.query(f"SELECT COUNT(*) as count FROM berlin_source_data.{table_name}", show_info=False)
    print(f"📊 Final rows: {final_count['count'].iloc[0]}")

print("\n🎉 SMART DB CONNECTOR CONSTRAINT TEST COMPLETED!")
print("\n💡 Key Findings:")
print("   ✅ Smart DB Connector populate() works with constraints")
print("   ✅ mode='append' preserves table constraints")
print("   ✅ Both Primary Key and Foreign Key constraints tested")
print("   💡 district_id preferred over district for FK references")
print("\n🚀 Our Smart DB Connector V3 successfully handles database constraints!")

🎯 STEP 4: FINAL VERIFICATION
📗 NeonDB Final Status:
🔒 Constraints after all operations: 4
   ✅ CHECK: 204800_1843224_1_not_null
   ✅ CHECK: 204800_1843224_2_not_null
   ✅ FOREIGN KEY: fk_smart_banks_test_6802_9949_district
   ✅ PRIMARY KEY: smart_banks_test_6802_9949_pkey
📊 Final rows: 3

📘 AWS Final Status:
🔒 Constraints after all operations: 4
   ✅ CHECK: 24602_57956_1_not_null
   ✅ CHECK: 24602_57956_2_not_null
   ✅ FOREIGN KEY: fk_smart_banks_test_6802_9949_district
   ✅ PRIMARY KEY: smart_banks_test_6802_9949_pkey
📊 Final rows: 3

🎉 SMART DB CONNECTOR CONSTRAINT TEST COMPLETED!

💡 Key Findings:
   ✅ Smart DB Connector populate() works with constraints
   ✅ mode='append' preserves table constraints
   ✅ Both Primary Key and Foreign Key constraints tested
   💡 district_id preferred over district for FK references

🚀 Our Smart DB Connector V3 successfully handles database constraints!
